In [ ]:
%tensorflow_version 1.x

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, Model
import matplotlib.pyplot as plt

from math import *
from tensorflow import keras

from tensorflow import keras
from keras.models import Model, load_model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam, SGD
from keras.layers import *
from keras.metrics import *
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from hyperopt import hp, STATUS_OK

In [ ]:
import h5py
import os
datadir="drive/MyDrive/pM10Kb_1KTest"
trainfile = h5py.File(os.path.join(datadir, 'train.h5'), 'r')
X_trainhalflife, X_trainpromoter, y_train, geneName_train = trainfile['data'], trainfile['promoter'], trainfile['label'], trainfile['geneName']
validfile = h5py.File(os.path.join(datadir, 'valid.h5'), 'r')
X_validhalflife, X_validpromoter, y_valid, geneName_valid = validfile['data'], validfile['promoter'], validfile['label'], validfile['geneName']
testfile = h5py.File(os.path.join(datadir, 'test.h5'), 'r')
X_testhalflife, X_testpromoter, y_test, geneName_test = testfile['data'], testfile['promoter'], testfile['label'], testfile['geneName']
leftpos = 9600
rightpos = 10400
XTrain = X_trainpromoter[:,leftpos:rightpos,:]
XVal = X_validpromoter[:,leftpos:rightpos,:]
XTest = X_testpromoter[:,leftpos:rightpos,:]
YTrain = y_train
YVal = y_valid
YTest = y_test

In [ ]:
layer_0 = Input(shape=XTrain.shape[1:],name='Input')

layer_1 = Conv1D(32,7,padding='same', kernel_initializer='glorot_normal',activation='relu',name='Conv1D_1')(layer_0)
layer_2 = MaxPool1D(4,name='MaxPool_1')(layer_1)

layer_3 = Conv1D(16,7,padding='same', kernel_initializer='glorot_normal',activation='relu',name='Conv1D_2')(layer_2)
layer_4 = MaxPool1D(2,name='MaxPool_2')(layer_3)

layer_5 = Conv1D(8,7,padding='same', kernel_initializer='glorot_normal',activation='relu',name='Conv1D_3')(layer_4)
layer_6 = MaxPool1D(2,name='MaxPool_3')(layer_5)

layer_7 = Flatten(name='Flatten_1')(layer_6)

layer_8 = Dense(8,name='Dense_1')(layer_7)
layer_9 = ReLU(name='ReLU_1')(layer_8)
layer_10 = Dropout(0.00099,name='dropout_1')(layer_9)

layer_11 = Dense(2,name='Dense_2')(layer_10)
layer_12 = ReLU(name='ReLU_2')(layer_11)
layer_13 = Dropout(0.00099,name='dropout_2')(layer_12)

layer_14 = Dense(50*8,kernel_regularizer=tf.keras.regularizers.l1(0.0001),name='Contextual_Weight')(layer_13)
layer_15 = Flatten(name='Flatten_2')(layer_6)
layer_16 = Multiply(name='Multiply')([layer_14,layer_15])

layer_17 = Dense(1,kernel_initializer='ones',use_bias=False,name='Sum')(layer_16)
layer_18 = Dense(1,name='Output')(layer_17)
model = Model(inputs=layer_0,outputs=layer_18)
model.layers[17].trainable = False
model.summary()
keras.utils.plot_model(model, "model_with_shape_info.png", show_shapes=True)

In [ ]:
model.compile(SGD(lr=0.0005, momentum=0.9),'mean_squared_error', metrics=['mean_squared_error'])
earlystop_cb = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')
check_cb = ModelCheckpoint('bestparams.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
result = model.fit(XTrain, YTrain, batch_size=128, shuffle="batch", epochs=100,
        validation_data=[XVal, YVal], callbacks=[earlystop_cb, check_cb])
mse_history = result.history['val_mean_squared_error']

In [ ]:
best_file = 'drive/MyDrive/Cell_Report_Core_CR_bestparams_3.h5'
model = load_model(best_file)
YTrain_Pred = model.predict(XTrain)
print('Correlation of training dataset: '+str(np.corrcoef(YTrain[:].transpose(),YTrain_Pred.transpose())[0,1]))
YVal_Pred = model.predict(XVal)
print('Correlation of training dataset: '+str(np.corrcoef(YVal[:].transpose(),YVal_Pred.transpose())[0,1]))
YTest_Pred = model.predict(XTest)
print('Correlation of training dataset: '+str(np.corrcoef(YTest[:].transpose(),YTest_Pred.transpose())[0,1]))